In [1]:
import sys 
sys.path.append("../")
from LIB.utils import * 

In [5]:
v="_0"
train=pd.read_csv("./data_proc/train_0.csv")
test=pd.read_csv("./data_proc/test_0.csv")
Taregt_name="retweet_count"

train[Taregt_name]=np.log1p(train[Taregt_name])

In [6]:
json_columns=['entities_urls',
 'entities_hashtags',
 'entities_user_mentions',
 'user_entities']

In [7]:
keys= ["id"]
Taregt_name="retweet_count"
not_used_cols=[ 'favorite_count', 'user_created_at', 'retweet_count',"in_reply_to_user_id","train"
               "in_reply_to_status_id","id","favorite_count","created_at","retweet_countpred","retweet_count_without_zeros",
              "entities_hashtags_text"]+json_columns
feautres_name=[ f for f in test.columns if f not in not_used_cols]
from sklearn.metrics import mean_absolute_error
def eval_metrics(y_ture,y_hat):
    return mean_absolute_error(y_ture,y_hat)

from LIB.lgbm_model import * 
params = {
            "objective" : "regression", 
            "metric" : "rmse",
            "max_depth":9,
            "feature_fraction":0.75,
            "bagging_fraction":0.75,
            "bagging_freq" : 2,
            "lambda" : 0.1,
            "num_threads" :30}


# params = {"objective" : "regression", 
#           "metric" : "rmse",
#           "max_depth": 8,  
#           "learning_rate" : 0.01,
#           "subsample" : 0.8, 
#           "colsample_bytree" : 0.9,
#           "num_threads":20}

In [8]:
LGBM= lightgbm_model( Train_df=train,
                        Test_df=test,
                        feval_metrics=eval_metrics,
                        Target_name=Taregt_name,
                        params=params,
                        feature_names=feautres_name,
                        keys=keys,
                        verbose_eval=50,
                        early_stopping_rounds=50,
                        num_boost_round=20000,
                        maximize=False,
                        test_size=0.15,
                        nbr_fold=10,
                        nbr_run=1)
train_pred_lgbm1,test_pred_lgbm=LGBM.lightgbm_Kfold()

test_pred_lgbm["retweet_count"]=np.expm1(test_pred_lgbm["retweet_count"])
test_pred_lgbm["retweet_count"]=test_pred_lgbm["retweet_count"].apply(lambda x : 0 if x<0 else x )



##################################################fold:1##################################################
run 0 train metrics :0.20476825056872328 val metrics : 0.24944000406096922

fold-1 train metrics :0.20476825056872328 val metrics : 0.24944000406096922
##################################################fold:2##################################################
run 0 train metrics :0.20554321113488333 val metrics : 0.2539742952367196

fold-2 train metrics :0.20554321113488333 val metrics : 0.2539742952367196
##################################################fold:3##################################################
run 0 train metrics :0.20126416460102683 val metrics : 0.24819185771378727

fold-3 train metrics :0.20126416460102683 val metrics : 0.24819185771378727
##################################################fold:4##################################################
run 0 train metrics :0.20413962293667584 val metrics : 0.24590539213475082

fold-4 train metrics :0.20413962293667584 

In [9]:
from LIB.xgb import * 
# best
params= {'min_child_weight': 10, 'eta': 0.04, 'colsample_bytree': 0.75, 'max_depth': 9,
            'subsample': 0.75, 'lambda': 3, 'nthread': 40, 'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:linear'}

# params= {'min_child_weight': 10, 'eta': 0.04, 'colsample_bytree': 0.75, 'max_depth': 9,
#             'subsample': 0.75, 'lambda': 3, 'nthread': 40, 'booster' : 'gbtree', 'silent': 1,
#             'eval_metric': 'rmse', 'objective': 'reg:linear'}

# params= {'min_child_weight': 20, 'eta': 0.03, 'colsample_bytree': 0.9, 'max_depth':10,
#             'subsample': 0.9,'lambda': 2.5, 'nthread': 40, 'booster' : 'gbtree', 'silent': 1,
#             'eval_metric': 'rmse', 'objective': 'reg:linear'}

XGBoost= Xgboost_model( Train_df=train,
                        Test_df=test,
                        feval_metrics=eval_metrics,
                        Target_name=Taregt_name,
                        params=params,
                        feature_names=feautres_name,
                        keys=keys,
                        verbose_eval=10,
                        early_stopping_rounds=100,
                        num_boost_round=20000,
                        maximize=False,
                        test_size=0.1,
                        nbr_fold=10,
                        nbr_run=1)
train_pred_xgboost,test_pred_xgboost=XGBoost.Xgboost_Kfold()
test_pred_xgboost["retweet_count"]=np.expm1(test_pred_xgboost["retweet_count"])
test_pred_xgboost["retweet_count"]=test_pred_xgboost["retweet_count"].apply(lambda x : 0 if x<0 else x )



##################################################fold:1##################################################
run 0 train metrics :0.15806657 val metrics : 0.24221319

fold-1 train metrics :0.15806657 val metrics : 0.24221319
##################################################fold:2##################################################
run 0 train metrics :0.15867966 val metrics : 0.24872243

fold-2 train metrics :0.15867966 val metrics : 0.24872243
##################################################fold:3##################################################
run 0 train metrics :0.11375469 val metrics : 0.24353318

fold-3 train metrics :0.11375469 val metrics : 0.24353318
##################################################fold:4##################################################
run 0 train metrics :0.13486819 val metrics : 0.23864982

fold-4 train metrics :0.13486819 val metrics : 0.23864982
##################################################fold:5##################################################
r

In [10]:
merge=test_pred_xgboost.copy()
merge.retweet_count=(merge.retweet_count*0.5+test_pred_lgbm.retweet_count*0.5)


test_pred_xgboost.retweet_count=test_pred_xgboost.retweet_count.round()
test_pred_xgboost.to_csv("sub/xgboost_Twi_model.csv",index=False)

test_pred_lgbm.retweet_count=test_pred_lgbm.retweet_count.round()
test_pred_lgbm.to_csv("sub/lgbm_Twi_model.csv",index=False)


merge.retweet_count=merge.retweet_count.round()
merge.to_csv("sub/merge_Twi_model.csv",index=False)

In [11]:
zip -r myfiles.zip ../twitter_solution/

SyntaxError: invalid syntax (<ipython-input-11-7e6f8b80c8ad>, line 1)